# Decoder DEV

This is a temporary notebook to use the interpreter to quickly develop a decoder for the data from the satellite in python. It will soon be deleted and replaced by the python scripts that do the decoding, which will also be incorporated into the raadpy library

In [123]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import os

# Define useful constants
data_dir        = '../../Data/RAW/'        # Filename with directories
BYTE            = 8                        # Byte length

ORBIT_STRUCT    = {
    'timestamp'     : 32,
    'temperature'   : 8,
    'rate0'         : 12,
    'rate1'         : 12,
    'rate2'         : 12,
    'rate3'         : 12,
    'ratev'         : 8,
    'hv_level'      : 12,
    'veto_level'    : 12,
    'id_bit'        : 1,
    'pps_active'    : 1,
    'suspended'     : 1,
    'power_on'      : 1,
    'scenario'      : 4,
}

In [124]:
# Take the orbit buffer and analyze the data
orbit_fname = data_dir+'/SD-915/Light1_2022-05-20_Buff1.dat'    # Filename with data
file = open(orbit_fname,'rb')                                   # Open the file in read binary mode

# Read the first 16 bytes
# for i in range(235):
#     entry = file.read(16)
# # entry = file.read(16)
# # entry = file.read(16)

# file.close()
# entry = file.read(16)
# entry

In [125]:
entry = file.read()[:16]
entry 

b'\x00\x00\x00\x00A\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08'

In [126]:
# We create a function that given a bytestring extracts the ith bit:
def get_bit(i:int,string):
    '''
    Gets the ith bit from a python bytestring from the left

    Input:
    i: int --> index (frist bit is 0)
    string --> the bytestring 
    '''

    # Which byte does the bit lie into?
    byte_idx    = i//BYTE               # Integer division
    assert(byte_idx < len(string))      # Assert that the index is in the bytestring
    byte        = string[byte_idx]      # Get the appropriate byte
    bit_idx     = i - byte_idx * BYTE   # Get the index within the byte

    # Get the ith bit
    return (byte & (1 << (BYTE - bit_idx - 1))) >> (BYTE - bit_idx - 1)

# Helper function to give the index of the nth bit in a Bytestring
def get_bit_idx(n:int):
    return BYTE - 1 - n%BYTE + (n//BYTE) * BYTE

# Get range of bits
def get_bits(start:int,length:int,string):
    '''
    Gets length bits after and including index start

    Input:
    start:  int --> Start index included
    length: int --> Length of bits to obtain
    string      --> The bytestring
    '''

    # Collect the bytes and add them up
    digit_sum = 0
    for i in range(start,start+length):
        digit_sum += 2**(i-start) * get_bit(get_bit_idx(i),string)

    return digit_sum

# Create a dictionary of orbits from a file
def get_dict(filename:str,struct=ORBIT_STRUCT):
    # Open the file in read binary mode
    file = open(filename,'rb')

    # Initialize the dictionary
    data = dict(zip(struct.keys(),[np.array(list()) for _ in range(len(ORBIT_STRUCT.keys()))]))

    while True:
        # Get the next 16 bits to an event
        event = file.read(16)

        # Check if you reach EOF
        if event == b'': break

        # Keep track of the number of bits read
        bits_read = 0

        # If not create an orbit
        for name,length in struct.items():
            data[name] = np.append(data[name],[get_bits(bits_read,length,event)])
            bits_read += length
        
    # Return the dictionary
    return data

In [127]:
data = get_dict(orbit_fname)
data.keys()

dict_keys(['timestamp', 'temperature', 'rate0', 'rate1', 'rate2', 'rate3', 'ratev', 'hv_level', 'veto_level', 'id_bit', 'pps_active', 'suspended', 'power_on', 'scenario'])

In [130]:
data

{'timestamp': array([0., 0., 0., ..., 0., 0., 0.]),
 'temperature': array([65., 70., 66., ..., 86., 88., 86.]),
 'rate0': array([ 0.,  0.,  0., ..., 13., 11.,  0.]),
 'rate1': array([ 0.,  0.,  0., ..., 14., 10.,  0.]),
 'rate2': array([ 0.,  0.,  0., ..., 15., 16.,  1.]),
 'rate3': array([ 0.,  0.,  0., ..., 12.,  9.,  1.]),
 'ratev': array([ 0.,  0.,  0., ..., 20., 14.,  1.]),
 'hv_level': array([   0.,    0., 3000., ..., 3855.,    0.,    0.]),
 'veto_level': array([   0.,    0., 1900., ..., 2319.,    0.,    0.]),
 'id_bit': array([0., 1., 0., ..., 1., 0., 1.]),
 'pps_active': array([0., 0., 0., ..., 0., 0., 0.]),
 'suspended': array([0., 0., 0., ..., 1., 1., 1.]),
 'power_on': array([1., 1., 0., ..., 0., 0., 0.]),
 'scenario': array([ 0.,  0.,  3., ..., 12., 12., 12.])}